<a href="https://colab.research.google.com/github/abhinavsathi/book_recommendation_engine/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from difflib import get_close_matches

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
user_counts = df_ratings['user'].value_counts()
isbn_counts = df_ratings['isbn'].value_counts()


df_ratings_filtered = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

df_ratings_filtered = df_ratings_filtered[df_ratings_filtered['isbn'].isin(isbn_counts[isbn_counts >= 100].index)]

In [ ]:
df = pd.merge(df_ratings_filtered, df_books, on='isbn')

print("After filtering: ratings rows =", len(df),
      "| unique books =", df['title'].nunique(),
      "| unique users =", df['user'].nunique())

book_user_matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)

book_user_sparse = csr_matrix(book_user_matrix.values)

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)

book_titles = list(book_user_matrix.index)

After filtering: ratings rows = 49517 | unique books = 673 | unique users = 888


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    if book not in book_user_matrix.index:
        matches = get_close_matches(book, book_user_matrix.index, n=1, cutoff=0.6)
        if matches:
            book = matches[0]
        else:
            return [book, []]

    query_vec = book_user_matrix.loc[book].values.reshape(1, -1)
    distances, indices = model.kneighbors(query_vec, n_neighbors=6)

    recs = []
    for i in range(1, 6):
        title_i = book_user_matrix.index[indices[0][i]]
        dist_i = float(distances[0][i])
        recs.append([title_i, dist_i])

    recs.reverse()

    return [book, recs]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉
